In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow
from tensorflow.keras import layers, models

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [3]:
print("Num GPUs Available: ", len(tensorflow.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [5]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [6]:
train_images.shape

(60000, 28, 28)

In [7]:
len(train_labels)

60000

In [8]:
test_images.shape

(10000, 28, 28)

In [9]:
train_images = train_images / 255.0

test_images = test_images / 255.0

In [10]:
def create_resnet():
    inputs = layers.Input(shape=(28, 28, 1))  # Adjust input shape for Fashion MNIST (grayscale)
   
    # Initial convolutional layer
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
 
    # First residual block
    for _ in range(2):
        residual = x
        x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(64, (3, 3), padding='same', activation=None)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, residual])
        x = layers.ReLU()(x)
 
    # Second residual block
    x = layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    for _ in range(2):
        residual = x
        x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(128, (3, 3), padding='same', activation=None)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, residual])
        x = layers.ReLU()(x)
 
    # Global average pooling and output
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(10, activation='softmax')(x)  # 10 classes for Fashion MNIST
 
    model = models.Model(inputs, outputs)
    return model

In [11]:
model = create_resnet()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 64)   640         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 28, 28, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 28, 28, 64)   36928       ['batch_normalization[0][0]']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
Train_images, Val_images, Train_labels, Val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [15]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [21]:
model.fit(Train_images, Train_labels, validation_data=(Val_images, Val_labels), epochs=50, batch_size=64, callbacks=[stop_early])

Epoch 1/50
750/750 [==============================] - 51s 19ms/step - loss: 0.4358 - accuracy: 0.8421 - val_loss: 0.4189 - val_accuracy: 0.8500
Epoch 2/50
750/750 [==============================] - 14s 18ms/step - loss: 0.2695 - accuracy: 0.9020 - val_loss: 0.2762 - val_accuracy: 0.8979
Epoch 3/50
750/750 [==============================] - 14s 18ms/step - loss: 0.2309 - accuracy: 0.9162 - val_loss: 0.2593 - val_accuracy: 0.9055
Epoch 4/50
750/750 [==============================] - 14s 18ms/step - loss: 0.2037 - accuracy: 0.9273 - val_loss: 0.5031 - val_accuracy: 0.8384
Epoch 5/50
750/750 [==============================] - 14s 18ms/step - loss: 0.1865 - accuracy: 0.9335 - val_loss: 0.2370 - val_accuracy: 0.9168
Epoch 6/50
750/750 [==============================] - 14s 18ms/step - loss: 0.1664 - accuracy: 0.9404 - val_loss: 0.2246 - val_accuracy: 0.9182
Epoch 7/50
750/750 [==============================] - 14s 18ms/step - loss: 0.1492 - accuracy: 0.9467 - val_loss: 0.2319 - val_accuracy:

In [23]:
model.save('models/resnet_basemodel.h5')

In [16]:
base_model = tf.keras.models.load_model('models/resnet_basemodel.h5')

In [17]:
import keras_tuner as kt

def model_builder(hp):
    # Load the pre-trained model
    base_model = load_model(r"D:\Pravin\MCW\Assignments\resnerArchi\models\resnet_basemodel.h5")

    # Compile the model with hyperparameters
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    if hp_optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    elif hp_optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

    base_model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return base_model


In [18]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='resnet_tuning',
    project_name='fasion-mnist'
)


Reloading Tuner from resnet_tuning\fasion-mnist\tuner0.json


In [19]:
tuner.search(Train_images, Train_labels, epochs=10, validation_data=(Val_images, Val_labels))

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best learning rate: {best_hps.get('learning_rate')}")
print(f"Best optimizer: {best_hps.get('optimizer')}")

Best learning rate: 0.0001
Best optimizer: rmsprop


In [44]:
hypertuned_model = tuner.hypermodel.build(best_hps)
hypertuned_model.fit(Train_images, Train_labels, epochs=50, validation_data=(Val_images, Val_labels), batch_size=64, callbacks=[stop_early])


Epoch 1/50
750/750 [==============================] - 17s 20ms/step - loss: 0.0103 - accuracy: 0.9971 - val_loss: 0.2617 - val_accuracy: 0.9387
Epoch 2/50
750/750 [==============================] - 15s 20ms/step - loss: 0.0041 - accuracy: 0.9995 - val_loss: 0.2686 - val_accuracy: 0.9402
Epoch 3/50
750/750 [==============================] - 15s 20ms/step - loss: 0.0022 - accuracy: 0.9999 - val_loss: 0.2764 - val_accuracy: 0.9406
Epoch 4/50
750/750 [==============================] - 15s 20ms/step - loss: 0.0014 - accuracy: 0.9999 - val_loss: 0.2851 - val_accuracy: 0.9391
Epoch 5/50
750/750 [==============================] - 15s 20ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.2928 - val_accuracy: 0.9414
Epoch 6/50
750/750 [==============================] - 15s 20ms/step - loss: 7.1293e-04 - accuracy: 1.0000 - val_loss: 0.3006 - val_accuracy: 0.9401
Epoch 7/50
750/750 [==============================] - 17s 22ms/step - loss: 4.9821e-04 - accuracy: 1.0000 - val_loss: 0.3112 - val_a

In [ ]:
hypertuned_model.save('models/resnet_hypertuned_model.h5')

In [29]:
hypertuned_model = tensorflow.keras.models.load_model(r"D:\Pravin\MCW\Assignments\resnerArchi\models\resnet_hypertuned_model.h5")

In [21]:
# Reshape the data to include the channel dimension (for grayscale images)
Train_images = Train_images.reshape(-1, 28, 28, 1)
Val_images = Val_images.reshape(-1, 28, 28, 1)

# Ensure the data is of type float32 and normalized
Train_images = Train_images.astype('float32')
Val_images = Val_images.astype('float32')


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
datagen = ImageDataGenerator(
    rotation_range=10,           # Slightly increase to account for diverse orientations
    width_shift_range=0.1,       # Smaller shifts since large shifts might cut off important details
    height_shift_range=0.1,      # Same as width shift for consistency
    shear_range=0.15,            # Slight reduction to avoid excessive distortion
    zoom_range=0.1,              # Zoom less to preserve spatial information
    fill_mode='nearest'          # Nearest fill mode works well for preserving structure
)

In [24]:
datagen.fit(Train_images)

In [25]:
augmented_model = hypertuned_model.fit(datagen.flow(Train_images, Train_labels, batch_size=64), epochs=50, validation_data=(Val_images, Val_labels), callbacks=[stop_early])


Epoch 1/50
750/750 [==============================] - 38s 21ms/step - loss: 0.4952 - accuracy: 0.9115 - val_loss: 0.6410 - val_accuracy: 0.9168
Epoch 2/50
750/750 [==============================] - 15s 21ms/step - loss: 0.3732 - accuracy: 0.9229 - val_loss: 0.5121 - val_accuracy: 0.9192
Epoch 3/50
750/750 [==============================] - 15s 21ms/step - loss: 0.3074 - accuracy: 0.9269 - val_loss: 0.4302 - val_accuracy: 0.9219
Epoch 4/50
750/750 [==============================] - 16s 21ms/step - loss: 0.2661 - accuracy: 0.9297 - val_loss: 0.3970 - val_accuracy: 0.9197
Epoch 5/50
750/750 [==============================] - 16s 22ms/step - loss: 0.2446 - accuracy: 0.9310 - val_loss: 0.3597 - val_accuracy: 0.9194
Epoch 6/50
750/750 [==============================] - 17s 23ms/step - loss: 0.2185 - accuracy: 0.9329 - val_loss: 0.3349 - val_accuracy: 0.9197
Epoch 7/50
750/750 [==============================] - 24s 32ms/step - loss: 0.1989 - accuracy: 0.9341 - val_loss: 0.2802 - val_accuracy:

In [27]:
augmented_model = hypertuned_model

In [28]:
augmented_model.save('models/resnet_augmented_model.h5')

In [40]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

#check the model performance
# Evaluate the base model
base_model_predictions = base_model.predict(test_images.reshape(-1, 28, 28, 1))
base_model_predictions = np.argmax(base_model_predictions, axis=1)
base_model_accuracy = accuracy_score(test_labels, base_model_predictions)
base_model_f1 = f1_score(test_labels, base_model_predictions, average='weighted')
base_model_recall = recall_score(test_labels, base_model_predictions, average='weighted')

# Evaluate the hypertuned model
hypertuned_model_predictions = hypertuned_model.predict(test_images.reshape(-1, 28, 28, 1))
hypertuned_model_predictions = np.argmax(hypertuned_model_predictions, axis=1)
hypertuned_model_accuracy = accuracy_score(test_labels, hypertuned_model_predictions)
hypertuned_model_f1 = f1_score(test_labels, hypertuned_model_predictions, average='weighted')
hypertuned_model_recall = recall_score(test_labels, hypertuned_model_predictions, average='weighted')

# Evaluate the augmented model
augmented_model_predictions = augmented_model.predict(test_images.reshape(-1, 28, 28, 1))
augmented_model_predictions = np.argmax(augmented_model_predictions, axis=1)
augmented_model_accuracy = accuracy_score(test_labels, augmented_model_predictions)
augmented_model_f1 = f1_score(test_labels, augmented_model_predictions, average='weighted')
augmented_model_recall = recall_score(test_labels, augmented_model_predictions, average='weighted')

print(f"Base Model - Accuracy: {base_model_accuracy}, F1 Score: {base_model_f1}, Recall: {base_model_recall}")
print(f"Hypertuned Model - Accuracy: {hypertuned_model_accuracy}, F1 Score: {hypertuned_model_f1}, Recall: {hypertuned_model_recall}")
print(f"Augmented Model - Accuracy: {augmented_model_accuracy}, F1 Score: {augmented_model_f1}, Recall: {augmented_model_recall}")


313/313 [==============================] - 2s 7ms/step
Base Model - Accuracy: 0.9166, F1 Score: 0.9170377265321873, Recall: 0.9166
Hypertuned Model - Accuracy: 0.9369, F1 Score: 0.9366843991753389, Recall: 0.9369
Augmented Model - Accuracy: 0.9312, F1 Score: 0.9319129805048401, Recall: 0.9312


In [41]:
import onnxruntime as ort
from sklearn.metrics import accuracy_score, f1_score, recall_score

# Load the ONNX model
onnx_model_path = 'models/resnet_augmented_model.onnx'
ort_session = ort.InferenceSession(onnx_model_path)

# Prepare the input data
onnx_input_name = ort_session.get_inputs()[0].name
onnx_input_data = test_images.reshape(-1, 28, 28, 1).astype('float32')

# Make predictions
onnx_predictions = ort_session.run(None, {onnx_input_name: onnx_input_data})[0]
onnx_predictions = np.argmax(onnx_predictions, axis=1)

# Evaluate the metrics
onnx_model_accuracy = accuracy_score(test_labels, onnx_predictions)
onnx_model_f1 = f1_score(test_labels, onnx_predictions, average='weighted')
onnx_model_recall = recall_score(test_labels, onnx_predictions, average='weighted')

print(f"ONNX Model - Accuracy: {onnx_model_accuracy}, F1 Score: {onnx_model_f1}, Recall: {onnx_model_recall}")

ONNX Model - Accuracy: 0.9312, F1 Score: 0.9319129805048401, Recall: 0.9312


In [44]:
import numpy as np
import tensorrt as trt
import cv2  # for loading image data
import tensorflow as tf  # for MNIST data handling
from sklearn.metrics import accuracy_score
import pycuda.driver as cuda
import pycuda.autoinit

# Load the TensorRT model (engine file)
def load_engine(trt_logger, engine_file):
    with open(engine_file, 'rb') as f:
        runtime = trt.Runtime(trt_logger)
        engine = runtime.deserialize_cuda_engine(f.read())
    return engine

# Prepare input data (example for MNIST dataset)
def prepare_input_data(image):
    # Reshape image to (1, 28, 28, 1) and normalize it
    image = cv2.resize(image, (28, 28))  # Resize if necessary
    image = np.expand_dims(image, axis=-1)  # Add channel dimension (28, 28, 1)
    image = np.expand_dims(image, axis=0)  # Add batch dimension (1, 28, 28, 1)
    image = image.astype(np.float32) / 255.0  # Normalize the image
    return image

# Run inference and get prediction
def run_inference(engine, input_data):
    context = engine.create_execution_context()

    # Allocate memory for input/output
    d_input = cuda.mem_alloc(input_data.nbytes)
    d_output = cuda.mem_alloc(input_data.nbytes)  # Adjust output size as needed

    bindings = [int(d_input), int(d_output)]
    
    # Transfer input data to GPU
    cuda.memcpy_htod(d_input, input_data)
    
    # Execute the model
    context.execute_v2(bindings)
    
    # Transfer output back to the host
    output_data = np.empty_like(input_data)
    cuda.memcpy_dtoh(output_data, d_output)

    return output_data

# Accuracy calculation
def calculate_accuracy(engine, dataset):
    correct_predictions = 0
    total_predictions = len(dataset)

    for image, label in dataset:
        input_data = prepare_input_data(image)
        output_data = run_inference(engine, input_data)

        # Assuming output_data has shape (1, 10), where 10 is the number of classes
        predicted_label = np.argmax(output_data)
        if predicted_label == label:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Load the model
trt_logger = trt.Logger(trt.Logger.WARNING)
engine_file = r'D:\Pravin\MCW\Assignments\resnerArchi\models\resnet_abi_model.trt'
engine = load_engine(trt_logger, engine_file)

# Load your dataset (here using MNIST from TensorFlow)

# Pair the images with labels
dataset = list(zip(test_images, test_labels))

# Calculate accuracy
accuracy = calculate_accuracy(engine, dataset)
print(f"Accuracy: {accuracy * 100:.2f}%")


AttributeError: 'NoneType' object has no attribute 'create_execution_context'